# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных
### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Загрузка файлов

In [2]:
data = []
for index in range(3):
    try:
        data.append(pd.read_csv(f'/datasets/geo_data_{index}.csv'))
    except:
        data.append(pd.read_csv(f'https://code.s3.yandex.net/datasets/geo_data_{index}.csv'))

### Просмотр первых строчек

In [3]:
for index in range(3):
    print(f"data_{index}")
    display(data[index].head())

data_0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


data_1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


data_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Просмотр основной информации

In [4]:
for index in range(3):
    print(f"\ndata_{index}")
    display(data[index].describe())
    data[index].info()


data_0


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

data_1


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

data_2


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Вывод раздела.** Данные трёх разных регионов находятся в трёх разный файлах, поэтому данные помещены в `list` для упрощения дальнейшей работы с ними.

## Обучение и проверка модели
Для дальнейшего анализа и обучения в столбеце `id` нет необходимости, а память надо экономить. Избавляюсь от `id` 

In [5]:
for index in range(3):
    data[index] = data[index].drop(columns=['id'])

Создаю листы `features` и `target` для хранения признаков и целевого признака

In [6]:
features = []
target = []
for d in data:
    features.append(d.drop(columns=['product']))
    target.append(d['product'])

Заранее подготавливаю листы для того, чтобы разбить признаки на тренировачные и валидационные

In [7]:
features_train = [None] * 3
features_valid = [None] * 3
target_train = [None] * 3
target_valid = [None] * 3

Разбиваю признаки на тренировачные и валидационныев соотношении 3:1

In [8]:
for index in range(3):
    features_train[index], features_valid[index], target_train[index], target_valid[index] = train_test_split(
        features[index], target[index], test_size=0.25, random_state=1
    )

Обучаю модель на тренировачных данных и делаю предсказания на валидационных данных. Предсказанные значения сохраняю в отдельном листе

In [9]:
# Лист для пресказанных данных
predicted_valid = []
# Линейная регрессия для всех регионов
model = LinearRegression()
for index in range(3):
    model.fit(features_train[index], target_train[index])
    # Добавляю результаты в список. Они нужны в будущем
    predicted_valid.append(model.predict(features_valid[index]))
    # Вывод результатов
    print(f"data_{index}:")
    print(f"\tpredict_mean: {predicted_valid[index].mean()}")
    print(f"\tRMSE: {mean_squared_error(target_valid[index], predicted_valid[index]) ** 0.5}")

data_0:
	predict_mean: 92.49262459838863
	RMSE: 37.74258669996437
data_1:
	predict_mean: 69.12040524285558
	RMSE: 0.8943375629130574
data_2:
	predict_mean: 94.9568304858529
	RMSE: 39.86671127773423


**Вывод раздела.** В данном разделе были выделены признаки от целевых признаков. Данные разделены в соотношении 3:1. Обучена модель и все её предсказания сохранены.

## Подготовка к расчёту прибыли
Нахожу бюджет для выработки одной скажины и объём сырья для выхода в плюс

In [10]:
# Все значения в рублях
# Общий бюджет
TOTAL_EXPENSES = 10_000_000_000
# Стоимость тыс. баррелей сырья
ONE_THOUSAND_BARREL_COST = 450_000
# Колличество исследуемых мест
RESEARCH_PLACES = 500
# Колличество необходимых мест для добычи
NEED_PLACES = 200

ONE_PLACE_CREATE_COST = TOTAL_EXPENSES / NEED_PLACES
print(f"Разработка одной скважины: {ONE_PLACE_CREATE_COST}")
VOLUME_FOR_PROFIT = ONE_PLACE_CREATE_COST / ONE_THOUSAND_BARREL_COST
print(f"Объём сырья (тыс. баррель) для выхода а плюс: {VOLUME_FOR_PROFIT}")

Разработка одной скважины: 50000000.0
Объём сырья (тыс. баррель) для выхода а плюс: 111.11111111111111


Нахожу средние значения объёма добытого сырья в каждом регионе

In [11]:
for index in range(3):
    print(f"data_{index} mean: {data[index]['product'].mean()}")

data_0 mean: 92.50000000000001
data_1 mean: 68.82500000000002
data_2 mean: 95.00000000000004


Среднего запаса сырья в каждом регионе меньше чем необходимо для получения прибыли

Пишу функцию рассчёта выручки

In [12]:
# Функция рассчёта прибыли
def profit(target, predict, count):
    # Сортируются значения предсказаний от большего к меньшему
    predict_sorted = predict.sort_values(ascending=False)
    # Собираются достоверные значения 
    selected = target[predict_sorted.index][:count]
    # Собранные значения суммируются и умножаются на стоимость 1 тыс. баррель сырья. Из них вычитается общий бюджет
    return ONE_THOUSAND_BARREL_COST * selected.sum() - TOTAL_EXPENSES

**Вывод раздела.** В данном разделе был подсчитан объём сырья для получения прибыли. Найдены средние значения добытого сырья в каждом регионе, а также составлена функция для рассчёта прибыли.

## Расчёт прибыли и рисков 
Сбрасываю индексы `target_valid`, так как они разбросаны

In [13]:
for index in range(3):
    target_valid[index] = target_valid[index].reset_index(drop=True)

Вычисляю доверительный интервал в 95%, среднюю прибыль и риск убытков для каждого региона

In [14]:
# Убираю случайность при повторном запуске программы
state = np.random.RandomState(1)

for index in range(3):
    # Подготавливаю лист со значениями
    values = []
    # Примению Bootstrap с 1000 выборок, чтобы найти распределение прибыли для каждого региона
    for i in range(1000):
        # Беру необходимую часть данных для одной выборки
        target_subsample = target_valid[index].sample(
            frac=RESEARCH_PLACES / target_valid[index].shape[0],
            replace=True,
            random_state=state
        )
        # predicted valid после предзказания значений получил тип данных ndarray. Перевожу в Series
        predict_subsample = pd.Series(predicted_valid[index])[target_subsample.index]
        # В values добавляю рассчитанную прибыль для лучших скважин
        values.append(profit(target_subsample, predict_subsample, NEED_PLACES))

    # Рассчитываю доверительный интервал 95%, среднюю прибыль и риск убытков
    values = pd.Series(values)
    mean = values.mean()
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    risk_of_losses_percent = (values < 0).sum() / values.shape[0] * 100
    
    print(f"data_{index}")
    print(f"\tСредняя прибыль: {mean}")
    print(f"\tДоверительный интервал (2.5%, 97.5%): ({lower}, {higher})")
    print(f"\tРиск убытков: {risk_of_losses_percent}%")

data_0
	Средняя прибыль: 464377670.35698986
	Доверительный интервал (2.5%, 97.5%): (-98453877.16982545, 1007469099.0475638)
	Риск убытков: 5.2%
data_1
	Средняя прибыль: 541636803.387977
	Доверительный интервал (2.5%, 97.5%): (125207614.25419222, 960960337.357717)
	Риск убытков: 0.4%
data_2
	Средняя прибыль: 420170371.349994
	Доверительный интервал (2.5%, 97.5%): (-109597815.00023474, 991155649.4251106)
	Риск убытков: 6.7%


**Вывод раздела.** В данном разделе был проводились рассчёты прибили и рисков для каждого региона с применением подготовленной модели машинного обучения. Для этого была применена техника Bootstrap с 1000 выборок. На основе результатов только второй регион (data_1) соответствует условию о том, что вероятность убытков ниже 2.5% и является регионом с наибольшей средней прибылью. Для бурения скважин лучше всего подходит второй регион.

**Вывод.** В данной работе были взяты данные «ГлавРосГосНефть». Целью было решить в каком регоне лучше всего бурить новую скважину. В ходе работы были выполнены следующие задачи:
+ изучены представленные данные;
+ выделены признаки и целевой признак;
+ данные разбиты на тренировачные и валидационные;
+ найден бюджет для выработки одной скажины и объём сырья для выхода в плюс;
+ написана функция рассчёта прибыли;
+ техникой Bootstrap найдены средняя прибыль, доверительный интервал и риски убытков;
+ на основе результатов был выбран второй регион для бурения скважин.

Главная цель проета была выполнена и «ГлавРосГосНефть» теперь знает, где бурить новую скважину.